<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/ESG_Abnormal_Return_MSCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Creat the Leader, Average, and Laggard Group

In [173]:
import pandas as pd
import numpy as np

In [174]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Benchmark.csv',index_col='Unnamed: 0')
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444
2,1,ISRG,Intuitive Surgical Inc,NASDAQ,Healthcare,3842,4319.500,5777.800,4778.800,6678.800,8263.800,20407.38309,24572.19197,40890.73727,54692.79043,68321.27748,8263.800,68321.27748,68321.57,11.684600,-5.306607,5963.7400,41776.876048,0.142752,38.0,0.293538,0.120955,-0.172583,5,3.0,2.0,5.32,-1.998444
3,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444
4,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444


In [175]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/ESG_MSCI1226.csv')
ESG=ESG[['Month','Ticker','ESG_Rating','ESG_Score']]
ESG.head()

,Month,Ticker,ESG_Rating,ESG_Score
0,1,A,AAA,8.8
1,1,AAL,CCC,0.7
2,1,AAON,A,5.8
3,1,AAP,BBB,4.6
4,1,AAPL,A,6.4


### Categorize by ESG_Score

In [176]:
Month_List = ESG['Month'].unique().tolist()
Month_List

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [177]:
df_ =pd.DataFrame()
for i in Month_List:
  df_temp = ESG[ESG.Month==i]
  df_temp['ESG_G']=pd.qcut(df_temp['ESG_Score'],5,labels=np.arange(1,6,1))
  df_ =df_.append(df_temp)
df_.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,Month,Ticker,ESG_Rating,ESG_Score,ESG_G
15560,11,ZBH,CCC,1.2,1
15561,11,ZBRA,A,6.1,4
15562,11,ZION,A,6.2,4
15563,11,ZTS,A,6.4,4
15564,11,ZUMZ,BBB,5.1,3


In [178]:
ESG =df_
ESG.head()

,Month,Ticker,ESG_Rating,ESG_Score,ESG_G
0,1,A,AAA,8.8,5
1,1,AAL,CCC,0.7,1
2,1,AAON,A,5.8,4
3,1,AAP,BBB,4.6,3
4,1,AAPL,A,6.4,4


In [179]:
df=df.merge(ESG, on=['Month','Ticker'])
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,AAA,8.8,5
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,BB,4.1,3
2,1,ISRG,Intuitive Surgical Inc,NASDAQ,Healthcare,3842,4319.500,5777.800,4778.800,6678.800,8263.800,20407.38309,24572.19197,40890.73727,54692.79043,68321.27748,8263.800,68321.27748,68321.57,11.684600,-5.306607,5963.7400,41776.876048,0.142752,38.0,0.293538,0.120955,-0.172583,5,3.0,2.0,5.32,-1.998444,B,2.7,1
3,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444,BB,3.3,2
4,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444,BBB,4.6,3


In [180]:
df['ESG_Group']=df['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,AAA,8.8,5,Leader
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,BB,4.1,3,Average
2,1,ISRG,Intuitive Surgical Inc,NASDAQ,Healthcare,3842,4319.500,5777.800,4778.800,6678.800,8263.800,20407.38309,24572.19197,40890.73727,54692.79043,68321.27748,8263.800,68321.27748,68321.57,11.684600,-5.306607,5963.7400,41776.876048,0.142752,38.0,0.293538,0.120955,-0.172583,5,3.0,2.0,5.32,-1.998444,B,2.7,1,Laggard
3,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444,BB,3.3,2,Average
4,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444,BBB,4.6,3,Average


# Step 2: Calculating ESG Portfolio Return Controlling Benchmark Group

In [181]:
temp = df.groupby(by=['Month','Benchmark','ESG_Group'],as_index=False)['Monthly_Return'].mean()
temp.columns=['Month','Benchmark','ESG_Group','ESG_Portfoio_Return']
temp.tail()

,Month,Benchmark,ESG_Group,ESG_Portfoio_Return
3381,11,5.54,Laggard,12.884160
3382,11,5.54,Leader,14.120359
3383,11,5.55,Average,11.515956
3384,11,5.55,Laggard,14.897547
3385,11,5.55,Leader,13.891487


In [182]:
df=df.merge(temp,on=['Month','Benchmark','ESG_Group'])
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group,ESG_Portfoio_Return
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.0,4831.0,4567.0,4748.0,4873.0,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.0,26461.74713,26461.71,11.637870,-3.223538,4652.4,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,AAA,8.8,5,Leader,3.357646
1,1,IBM,International Business Machines Corp,NYSE,Technology,7373,14262.0,18246.0,17594.0,16795.0,20841.0,133506.55550,157832.34690,142034.91320,101448.13460,118710.84430,20841.0,118710.84430,118908.30,8.191835,7.229185,17547.6,130706.558900,0.134252,74.0,0.295393,0.175561,-0.119832,5,3.0,2.0,5.32,-1.998444,AA,8.5,5,Leader,3.357646
2,1,KO,Coca-Cola Co,NYSE,Consumer Non-Cyclicals,2086,25554.0,23062.0,17072.0,16981.0,18981.0,186832.40230,178815.29740,195479.41650,201545.93310,237146.59770,18981.0,237146.59770,237146.60,5.952382,5.510390,20330.0,199963.929400,0.101668,21.0,0.255102,0.080039,-0.175063,5,3.0,2.0,5.32,-1.998444,AA,7.6,5,Leader,3.357646
3,1,PEP,PepsiCo Inc,NASDAQ,Consumer Non-Cyclicals,2086,11923.0,11095.0,10889.0,14518.0,14786.0,145568.52960,150058.57980,170543.43140,155950.03260,190577.47760,14786.0,190577.47760,190577.40,11.390850,3.914547,12642.2,162539.610200,0.077779,21.0,0.255102,0.077585,-0.177517,5,3.0,2.0,5.32,-1.998444,AA,7.9,5,Leader,3.357646
4,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.0,8718.0,8947.0,8233.0,8405.0,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.0,46454.07146,46588.23,9.389818,-6.071341,8374.2,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,BB,4.1,3,Average,-5.017810


# Step 3: Calculating Abonormal Return

In [183]:
df['ESG_Abnormal_Return']=df['ESG_Portfoio_Return']-df['Portfoio_Return']
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group,ESG_Portfoio_Return,ESG_Abnormal_Return
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.0,4831.0,4567.0,4748.0,4873.0,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.0,26461.74713,26461.71,11.637870,-3.223538,4652.4,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,AAA,8.8,5,Leader,3.357646,5.356090
1,1,IBM,International Business Machines Corp,NYSE,Technology,7373,14262.0,18246.0,17594.0,16795.0,20841.0,133506.55550,157832.34690,142034.91320,101448.13460,118710.84430,20841.0,118710.84430,118908.30,8.191835,7.229185,17547.6,130706.558900,0.134252,74.0,0.295393,0.175561,-0.119832,5,3.0,2.0,5.32,-1.998444,AA,8.5,5,Leader,3.357646,5.356090
2,1,KO,Coca-Cola Co,NYSE,Consumer Non-Cyclicals,2086,25554.0,23062.0,17072.0,16981.0,18981.0,186832.40230,178815.29740,195479.41650,201545.93310,237146.59770,18981.0,237146.59770,237146.60,5.952382,5.510390,20330.0,199963.929400,0.101668,21.0,0.255102,0.080039,-0.175063,5,3.0,2.0,5.32,-1.998444,AA,7.6,5,Leader,3.357646,5.356090
3,1,PEP,PepsiCo Inc,NASDAQ,Consumer Non-Cyclicals,2086,11923.0,11095.0,10889.0,14518.0,14786.0,145568.52960,150058.57980,170543.43140,155950.03260,190577.47760,14786.0,190577.47760,190577.40,11.390850,3.914547,12642.2,162539.610200,0.077779,21.0,0.255102,0.077585,-0.177517,5,3.0,2.0,5.32,-1.998444,AA,7.9,5,Leader,3.357646,5.356090
4,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.0,8718.0,8947.0,8233.0,8405.0,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.0,46454.07146,46588.23,9.389818,-6.071341,8374.2,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,BB,4.1,3,Average,-5.017810,-3.019366


# Step 4: Visualizing Abnormal Return Monthly

In [184]:
import plotly.express as px
fig = px.box(df, x="ESG_Group", y="ESG_Abnormal_Return",color='ESG_Group',animation_frame='Month',animation_group='Ticker')
fig.show()

In [185]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['ESG_Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.398857,1.930110,863,29.376862,0.065702
1,1,Laggard,0.232098,5.047530,289,17.000000,0.296914
2,1,Leader,1.057775,4.310388,262,16.186414,0.266297
3,2,Average,-0.327588,2.215982,863,29.376862,0.075433
4,2,Laggard,1.420770,5.268137,289,17.000000,0.309890


In [186]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

In [187]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [188]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

# Step 5: Conduct Two-group t-test for each month

In [189]:
pip install pingouin

In [190]:
import pingouin as pg


Two-Sample T-Test

https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

In [191]:
temp.head()

,Month,Benchmark,ESG_Group,ESG_Portfoio_Return
0,1,1.11,Average,-9.777180
1,1,1.11,Laggard,10.741543
2,1,1.11,Leader,-11.817850
3,1,1.12,Average,-0.239942
4,1,1.12,Laggard,-6.472280


In [192]:
temp=temp.pivot(index=['Month','Benchmark'], columns='ESG_Group', values='ESG_Portfoio_Return').reset_index()
temp.head()

ESG_Group,Month,Benchmark,Average,Laggard,Leader
0,1,1.11,-9.777180,10.741543,-11.817850
1,1,1.12,-0.239942,-6.472280,0.437899
2,1,1.13,-12.043968,-1.786322,NaN
3,1,1.14,-6.317411,1.566678,NaN
4,1,1.15,1.231310,6.393337,NaN


### Test Monthly the Leaders' Abornormal Return >0?

In [193]:
temp.isna().sum()

ESG_Group
Month          0
Benchmark      0
Average        0
Laggard      173
Leader       191
dtype: int64

In [194]:
# temp=temp.dropna()

In [195]:
Leader = temp[temp['Month']==1]['Leader']
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-2.558745,101,two-sided,0.011989,"[-2.89, -0.37]",0.253354,2.412,0.71715


In [196]:
df_ = pd.DataFrame()
Month_List =temp.Month.unique().tolist()
for i in Month_List:
  Leader = temp[temp.Month==i]['Leader']
  t_test = pg.ttest(Leader, 0)
  t_test['Month']=i
  df_=df_.append(t_test)

In [197]:
df_

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,-2.558745,101,two-sided,1.198859e-02,"[-2.89, -0.37]",0.253354,2.412,0.717150,1
T-test,-14.330054,110,two-sided,6.520941e-27,"[-11.95, -9.05]",1.360149,5.608e+23,1.000000,2
T-test,-11.442251,110,two-sided,1.916768e-20,"[-22.7, -15.99]",1.086050,2.547e+17,1.000000,3
T-test,14.344063,101,two-sided,4.157520e-26,"[19.81, 26.17]",1.420274,9.074e+22,1.000000,4
T-test,9.936107,103,two-sided,1.026298e-16,"[7.82, 11.72]",0.974315,5.852e+13,1.000000,5
T-test,2.302716,103,two-sided,2.330551e-02,"[0.22, 2.92]",0.225800,1.35,0.625997,6
T-test,7.475134,103,two-sided,2.632961e-11,"[5.07, 8.73]",0.732997,3.303e+08,1.000000,7
T-test,-5.875719,106,two-sided,4.919420e-08,"[-5.37, -2.66]",0.568027,2.369e+05,0.999944,9
T-test,2.145088,103,two-sided,3.429647e-02,"[0.12, 3.12]",0.210343,0.974,0.565588,10
T-test,13.181745,109,two-sided,2.649375e-24,"[14.48, 19.6]",1.256830,1.54e+21,1.000000,11


In [198]:
df_2 = pd.DataFrame()
Month_List =temp.Month.unique().tolist()
for i in Month_List:
  Leader = temp[temp.Month==i]['Leader']
  Laggard = temp[temp.Month==i]['Laggard']
  t_test = pg.ttest(Leader, Laggard,paired=False)
  t_test['Month']=i
  df_2=df_2.append(t_test)

In [199]:
df_2

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,2.439902,210.512491,two-sided,0.015519,"[0.42, 3.96]",0.334135,2.39,0.679049,1
T-test,-2.084516,212.530483,two-sided,0.038308,"[-3.89, -0.11]",0.280630,1.123,0.544610,2
T-test,1.285168,203.476720,two-sided,0.200194,"[-1.45, 6.88]",0.172678,0.319,0.248195,3
T-test,-0.818034,203.888211,two-sided,0.414293,"[-6.43, 2.66]",0.113948,0.208,0.128644,4
T-test,0.817665,199.859905,two-sided,0.414522,"[-1.47, 3.55]",0.113051,0.206,0.128936,5
T-test,0.340702,203.230807,two-sided,0.733680,"[-1.46, 2.07]",0.046936,0.158,0.063351,6
T-test,2.601499,178.760060,two-sided,0.010059,"[0.69, 5.04]",0.359821,3.475,0.743141,7
T-test,1.215293,194.677507,two-sided,0.225726,"[-0.64, 2.71]",0.166308,0.299,0.226823,9
T-test,1.201595,207.999594,two-sided,0.230887,"[-0.83, 3.4]",0.165815,0.296,0.223199,10
T-test,0.224314,215.697652,two-sided,0.822725,"[-3.12, 3.92]",0.030371,0.151,0.055726,11


# Overall test

In [200]:
Leader =temp['Leader']
Laggard =temp['Laggard']

In [202]:
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,4.490663,1058,two-sided,0.000008,"[1.26, 3.22]",0.137995,732.896,0.994242


In [203]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,1.082683,2129.363389,two-sided,0.279072,"[-0.61, 2.11]",0.046866,0.087,0.191282
